In [1]:
from collections import defaultdict
import pandas as pd
import numpy
import scipy.optimize
numpy.random.seed(4234)
from math import log, exp
data = pd.read_csv('train.csv',header=0)
# print(list(data))
song_data = pd.read_csv('songs.csv',header=0)
# print(list(song_data))
members= pd.read_csv('members.csv',header=0)
# print list(members)

In [2]:
#Random Samples
# from random import randrange, uniform
# # from sets import Set
I = set()
# while (len(I) < 300000):
#     x = randrange(0, len(data))
#     I.add(x)

for i in range(300000) :
    I.add(i)

# X =[(data['msno'][i],data['song_id'][i]) for i in range(len(data))]
# Y = [data['target'][i] for i in range(len(data))]

In [3]:
# Data sample
X_orig = [(data['msno'][i], data['song_id'][i]) for i in I]
Y_orig = [data['target'][i] for i in I]

X = X_orig[:int(0.6*len(X_orig))]
Y = Y_orig[:int(0.6*len(X_orig))]

X_valid = X_orig[int(0.6*len(X_orig)):int(0.8*len(X_orig))]
Y_valid = Y_orig[int(0.6*len(X_orig)):int(0.8*len(X_orig))]

X_test = X_orig[int(0.8*len(X_orig)):]
Y_test = Y_orig[int(0.8*len(X_orig)):]

In [4]:
# Data Stats
from collections import defaultdict
dd = defaultdict(int)
ss = defaultdict(int)
for i in range(len(X)):
    dd[X[i][0]]+=1
    ss[X[i][1]]+=1
avg = 0
for u in dd:
    avg += dd[u]
avg = avg * 1.0 / len(dd)
print('num_users: ', len(dd))
print('num_songs: ', len(ss))
print(avg)
print(sum(Y))

num_users:  9204
num_songs:  41634
19.55671447196871
134683


In [5]:
songs =[(song_data['song_id'][i],song_data['song_length'][i],song_data['genre_ids'][i],
     song_data['artist_name'][i],song_data['composer'][i],song_data['lyricist'][i],
    song_data['language'][i],) for i in range(len(song_data))]

In [6]:
## User User Similarity based on song

song_list = defaultdict(set)
user_list=dict(defaultdict(str))

for d in range(len(X)):
    user = X[d][0]
    song = X[d][1]
    if Y[d]==1:
        song_list[song].add(user)

for i in song_list:
    s=song_list[i]
    for u1 in s:
        for u2 in s:
            if u1!=u2:
                if u1 in user_list:
                    if u2 in user_list[u1]:
                        user_list[u1][u2] +=1
                    else:
                        user_list[u1][u2]=1
                else:
                    user_list[u1]=defaultdict(str)
                    user_list[u1][u2]=1
                    




In [7]:
## Song Song Similarity based on users
users_songlist=defaultdict(set)
songs_songs_list=dict(defaultdict(str))

for d in range(len(X)):
    user = X[d][0]
    song = X[d][1]
    if Y[d]==1:
        users_songlist[user].add(song)

for i in users_songlist:
    u=users_songlist[i]
    for s1 in u:
        for s2 in u:
            if s1!=s2:
                if s1 in songs_songs_list:
                    if s2 in songs_songs_list[s1]:
                        songs_songs_list[s1][s2] +=1
                    else:
                        songs_songs_list[s1][s2]=1
                else:
                    songs_songs_list[s1]=defaultdict(str)
                    songs_songs_list[s1][s2]=1




In [8]:
#Generate Dictionaries for songs

genre_list = defaultdict(int)
song_genre_list = dict(defaultdict(int))
artist_list = defaultdict(int)
language_list = defaultdict(int)
composer_list = defaultdict(int)
song_artist_list = defaultdict(str)
song_language_list = defaultdict(str)
song_composer_list = defaultdict(str)

for i in range(len(song_data)):
    s = song_data['song_id'][i]

    if s not in song_genre_list:
        song_genre_list[s] = defaultdict(int)
    if not pd.isnull(song_data.loc[i,'genre_ids']):
        genlist = song_data['genre_ids'][i].split('|')
        #print(genlist)
        for j in range(len(genlist)):
            song_genre_list[s][genlist[j]] = 1
            genre_list[genlist[j]] += 1

    song_artist_list[s] = song_data['artist_name'][i]
    artist_list[song_data['artist_name'][i]] += 1

    if not pd.isnull(song_data.loc[i,'language']) and song_data['language'][i] != -1:
        song_language_list[s] = song_data['language'][i]
        language_list[song_data['language'][i]] += 1

    if  not pd.isnull(song_data.loc[i,'composer']):
        song_composer_list[s] = song_data['composer'][i]
        composer_list[song_data['composer'][i]] += 1



In [9]:
#Generate Dictionaries for members

gender_list = defaultdict(int)
city_list = defaultdict(int)
bd_list = defaultdict(int)
user_gender_list = defaultdict(str)
user_city_list = defaultdict(int)
user_bd_list = defaultdict(int)

for i in range(len(members)):
    u = members['msno'][i]
    
    if not pd.isnull(members.loc[i,'gender']):
        user_gender_list[u] = members['gender'][i]
        gender_list[members['gender'][i]] += 1
    
    user_city_list[u] = members['city'][i]
    city_list[members['city'][i]] += 1
    
    if members['bd'][i] != 0:
        user_bd_list[u] = members['bd'][i]
        bd_list[members['bd'][i]] += 1


In [10]:
#Generate Threshhold Dictionaries
gender_genre_list = dict(defaultdict(int))
gender_artist_list = dict(defaultdict(int))
gender_composer_list = dict(defaultdict(int))
bd_genre_list = dict(defaultdict(int))
bd_artist_list = dict(defaultdict(int))
city_genre_list = dict(defaultdict(int))
city_language_list = dict(defaultdict(int))
city_artist_list = dict(defaultdict(int))

for i in range(len(X)):
    u = X[i][0]
    s = X[i][1]
    
    if u in user_gender_list:
        if user_gender_list[u] not in gender_genre_list:
            gender_genre_list[user_gender_list[u]] = defaultdict(int)
        if s in song_genre_list:
            for j in song_genre_list[s]:
                gender_genre_list[user_gender_list[u]][j] += 1
        if user_gender_list[u] not in gender_composer_list:
            gender_composer_list[user_gender_list[u]] = defaultdict(int)
        if s in song_composer_list:
            gender_composer_list[user_gender_list[u]][song_composer_list[s]] += 1
        if user_gender_list[u] not in gender_artist_list:
            gender_artist_list[user_gender_list[u]] = defaultdict(int)
        gender_artist_list[user_gender_list[u]][song_artist_list[s]] += 1
    
    if u in user_bd_list:
        if user_bd_list[u] not in bd_genre_list:
            bd_genre_list[user_bd_list[u]] = defaultdict(int)
        if s in song_genre_list:
            for j in song_genre_list[s]:
                bd_genre_list[user_bd_list[u]][j] += 1
        if user_bd_list[u] not in bd_artist_list:
            bd_artist_list[user_bd_list[u]] = defaultdict(int)
        bd_artist_list[user_bd_list[u]][song_artist_list[s]] += 1
    
    if user_city_list[u] not in city_genre_list:
        city_genre_list[user_city_list[u]] = defaultdict(int)
    if s in song_genre_list:
        for j in song_genre_list[s]:
            city_genre_list[user_city_list[u]][j] += 1
    if user_city_list[u] not in city_language_list:
        city_language_list[user_city_list[u]] = defaultdict(int)
    city_language_list[user_city_list[u]][song_language_list[s]] += 1
    if user_city_list[u] not in city_artist_list:
        city_artist_list[user_city_list[u]] = defaultdict(int)
    city_artist_list[user_city_list[u]][song_artist_list[s]] += 1


In [11]:
#Generate User Artist similarity

artist_userlist=defaultdict(set)
sim_users_art=dict(defaultdict(str))

for d in range(len(X)):
    user = X[d][0]
    song = X[d][1]
    if Y[d]==1:
        artist= song_artist_list[song]
        artist_userlist[artist].add(user)

for i in artist_userlist:
    a=artist_userlist[i]
    for u1 in a:
        for u2 in a:
            if u1!=u2:
                if u1 in sim_users_art:
                    if u2 in sim_users_art[u1]:
                        sim_users_art[u1][u2] +=1
                    else:
                        sim_users_art[u1][u2]=1
                else:
                    sim_users_art[u1]=defaultdict(str)
                    sim_users_art[u1][u2]=1

In [12]:
# User Language Similarity
language_userlist=defaultdict(set)
sim_users_lang=dict(defaultdict(str))

for d in range(len(X)):
    user = X[d][0]
    song = X[d][1]
    lang= song_language_list[song]
    if Y[d]==1:
        language_userlist[lang].add(user)



for i in language_userlist:
    l=language_userlist[i]
    for u1 in l:
        for u2 in l:
            if u1!=u2:
                if u1 in sim_users_lang:
                    if u2 in sim_users_lang[u1]:
                        sim_users_lang[u1][u2] +=1
                    else:
                        sim_users_lang[u1][u2]=1
                else:
                    sim_users_lang[u1]=defaultdict(str)
                    sim_users_lang[u1][u2]=1

In [13]:
#User Genre Similarity 

genre_userlist=defaultdict(set)
sim_users_genre=dict(defaultdict(str))

for d in range(len(X)):
    user = X[d][0]
    song = X[d][1]
    if Y[d]==1:
        if song in song_genre_list:
            genres= song_genre_list[song]
            for g in genres:
                    genre_userlist[g].add(user)
    

for i in genre_userlist:
    g=genre_userlist[i]
    for u1 in g:
        for u2 in g:
            if u1!=u2:
                if u1 in sim_users_genre:
                    if u2 in sim_users_genre[u1]:
                        sim_users_genre[u1][u2] +=1
                    else:
                        sim_users_genre[u1][u2]=1
                else:
                    sim_users_genre[u1]=defaultdict(str)
                    sim_users_genre[u1][u2]=1    



            

# print(len(sim_users_genre))

In [14]:
gender_genre_sorted = defaultdict(list)
gender_artist_sorted = defaultdict(list)
gender_composer_sorted = defaultdict(list)
bd_genre_sorted = defaultdict(list)
bd_artist_sorted = defaultdict(list)
city_genre_sorted = defaultdict(list)
city_language_sorted = defaultdict(list)
city_artist_sorted = defaultdict(list)

for g in gender_genre_list:
    gender_genre_sorted[g] = sorted(gender_genre_list[g].items(), key=lambda x:x[1], reverse=True)
for g in gender_artist_list:
    gender_artist_sorted[g] = sorted(gender_artist_list[g].items(), key=lambda x:x[1], reverse=True)
for g in gender_composer_list:
    gender_composer_sorted[g] = sorted(gender_composer_list[g].items(), key=lambda x:x[1], reverse=True)
for b in bd_genre_list:
    bd_genre_sorted[b] = sorted(bd_genre_list[b].items(), key=lambda x:x[1], reverse=True)
for b in bd_artist_list:
    bd_artist_sorted[b] = sorted(bd_artist_list[b].items(), key=lambda x:x[1], reverse=True)
for c in city_genre_list:
    city_genre_sorted[c] = sorted(city_genre_list[c].items(), key=lambda x:x[1], reverse=True)
for c in city_language_list:
    city_language_sorted[c] = sorted(city_language_list[c].items(), key=lambda x:x[1], reverse=True)
for c in city_artist_list:
    city_artist_sorted[c] = sorted(city_artist_list[c].items(), key=lambda x:x[1], reverse=True)

In [15]:
def similarity_check(A,B,x,y):
    flag=False
    if x in A:
        for u in A[x]:
            if u in B[y]:
                return 1
    if flag==False:
            return 0 
        
    return 0

def threshold_check(dictionary,uval,sval,thres):
    ctr = 0
    for i in range(len(dictionary[uval])):
        ctr += 1
        if sval == dictionary[uval][i][0]:
            return 1
        if ctr > thres*len(dictionary[uval]):
            break
    return 0

In [16]:
import math
#Extract Features
gl_count = 0
city_dict=dict(zip(city_list, range(len(city_list))))
bd_dict=dict(zip(bd_list, range(len(bd_list))))
gender_dict=dict(zip(gender_list, range(len(gender_list))))
genre_dict=dict(zip(genre_list, range(len(genre_list))))
language_dict= dict(zip(language_list, range(len(language_list))))


def feature1(datum):
    global gl_count
    gl_count+=1
    if gl_count % 1000 == 0:
        print(gl_count)
    feat1=[0]*(len(city_list)+len(bd_list)+len(gender_list)+len(genre_list)+len(language_list))
    ##print(feat1)
    user=datum[0]
    song=datum[1]
    
    curr_city=city_dict[user_city_list[user]]
    feat1[curr_city]=1
    if user in user_bd_list:
        curr_bd=bd_dict[user_bd_list[user]]
        feat1[len(city_list)+curr_bd]=1
    if user in user_gender_list:
        curr_gender=gender_dict[user_gender_list[user]]
        feat1[len(city_list)+len(bd_list)+curr_gender]=1
    if song in song_genre_list:
        for i in song_genre_list[song]:
            curr_genre=genre_dict[i]
            feat1[len(city_list)+len(bd_list)+len(gender_list)+curr_genre]=1
    if song in song_language_list:
#         print(song)
#         print(song_language_list[song])
        if song_language_list[song] not in [-1, ''] and not math.isnan(song_language_list[song]):
            curr_lang=language_dict[song_language_list[song]]
            feat1[len(city_list)+len(bd_list)+len(gender_list)+len(genre_list)+curr_lang]=1
        
     ##Append for user-user similarity based on song
    val=similarity_check(user_list,song_list,user,song)
#     print(val)
#     print("user-user sim",val)
    feat1.append(val) 
    
    
    ##Append for song-song similarity based on user
    val=similarity_check(songs_songs_list,users_songlist,song,user)
#     print("song-song sim",val)
    feat1.append(val)
    
    ##Append for user user similarity based on Artist
    if song in song_artist_list:
        artist=song_artist_list[song]
        val=similarity_check(sim_users_art,artist_userlist,user,artist)
#         print("user-art sim",val)
        feat1.append(val)
    else:
        feat1.append(0)
    
    ##Append for user user similarity based on Genere
    if song in song_genre_list:
        flag=False
        for g in song_genre_list[song]:
            val=similarity_check(sim_users_genre,genre_userlist,user,g)
            if val==1:
                feat1.append(1)
                flag=True
                break
        if flag==False:
            feat1.append(0)
    else:
        feat1.append(0)
    
    ##Append for user-user similarity based on Language
    if song in song_language_list:
        lang=song_language_list[song]
        val=similarity_check(sim_users_lang,language_userlist,user,lang)
        feat1.append(val)
        
    else:
        feat1.append(0)
        
    
    feat1.append(1)
    return feat1
    
    ## Append Gender-Genre Threshold Value
    if user in user_gender_list and song in song_genre_list:
        flag = False
        for g in song_genre_list[song]:
            val = threshold_check(gender_genre_sorted,user_gender_list[user],g,0.5)
            if val == 1:
                feat1.append(1)
                flag = True
                break
        if flag == False:
            feat1.append(0)
    else:
        feat1.append(0)
    
    ## Append Gender-Artist Threshold
    if user in user_gender_list and song in song_artist_list:
        val = threshold_check(gender_artist_sorted,user_gender_list[user],song_artist_list[song],0.5)
        feat1.append(val)
    else:
        feat1.append(0)
     
    ## Append BD-Genre Threshold Value
    if user in user_bd_list and song in song_genre_list:
        flag = False
        for g in song_genre_list[song]:
            val = threshold_check(bd_genre_sorted,user_bd_list[user],g,0.5)
            if val == 1:
                feat1.append(1)
                flag = True
                break
        if flag == False:
            feat1.append(0)
    else:
        feat1.append(0)
    
    ## Append BD-Artist Threshold
    if user in user_bd_list and song in song_artist_list:
        val = threshold_check(bd_artist_sorted,user_bd_list[user],song_artist_list[song],0.5)
        feat1.append(val)
    else:
        feat1.append(0)
    
    ## Append City-Genre Threshold Value
    if user in user_city_list and song in song_genre_list:
        flag = False
        for g in song_genre_list[song]:
            val = threshold_check(city_genre_sorted,user_city_list[user],g,0.5)
            if val == 1:
                feat1.append(1)
                flag = True
                break
        if flag == False:
            feat1.append(0)
    else:
        feat1.append(0)
    
    ## Append City-Language Threshold
    if user in user_city_list and song in song_language_list:
        val = threshold_check(city_language_sorted,user_city_list[user],song_language_list[song],0.5)
        feat1.append(val)
    else:
        feat1.append(0)
    
    ## Append City-Artist Threshold
    if user in user_city_list and song in song_artist_list:
        val = threshold_check(city_artist_sorted,user_city_list[user],song_artist_list[song],0.5)
        feat1.append(val)
    else:
        feat1.append(0)
    
    feat1.append(1)
    return feat1
    
    
# genre_userlist=defaultdict(set)
# sim_users_genre=dict(defaultdict(str))   


In [18]:
gl_count = 0
X_train_feat=[feature1(d) for d in X]
gl_count = 0
X_valid_feat=[feature1(d) for d in X_valid]
gl_count = 0
X_test_feat=[feature1(d) for d in X_test]

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [19]:
gl_count = 0
X_valid_feat=[feature1(d) for d in X_valid]
gl_count = 0
X_test_feat=[feature1(d) for d in X_test]

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000


In [20]:
def inner(x,y):
  return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
  return 1.0 / (1 + exp(-x))

##################################################
# Logistic regression by gradient ascent         #
##################################################

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
  loglikelihood = 0
  for i in range(len(X)):
    logit = inner(X[i], theta)
    loglikelihood -= log(1 + exp(-logit))
    if not y[i]:
      loglikelihood -= logit
  for k in range(len(theta)):
    loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
  return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
  dl = [0]*len(theta)
  for i in range(len(X)):
    logit = inner(X[i], theta)
    for k in range(len(theta)):
      dl[k] += X[i][k] * (1 - sigmoid(logit))
      if not y[i]:
        dl[k] -= X[i][k]
  for k in range(len(theta)):
    dl[k] -= lam*2*theta[k]
  return numpy.array([-x for x in dl])

In [21]:
##################################################
# Train                                          #
##################################################
weights0 = numpy.random.normal(size=(1,len(X_train_feat[0])))
#print(weights0)
def train(lam):
  theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, weights0,\
            fprime, pgtol = 10, args = (X_train_feat, Y, lam))
  return theta

##################################################
# Predict                                        #
##################################################

def performance(theta, X, y):
  scores = [inner(theta,x) for x in X]
  predictions = [s > 0 for s in scores]
  correct = [(a==b) for (a,b) in zip(predictions,y)]
  acc = sum(correct) * 1.0 / len(correct)
  return acc

##################################################
# Validation pipeline                            #
##################################################

lam = 1.0

theta = train(lam)
# print(X)
acc_valid = performance(theta, X_valid_feat, Y_valid)
acc_test = performance(theta, X_test_feat, Y_test)
print(acc_valid)
print(acc_test)

KeyboardInterrupt: 

In [ ]:
print(len(Y_valid))